# ECONOVISION

Neste projeto, desenvolvemos um dashboard que utiliza dados de indicadores socioeconômicos para criar gráficos e análises de regressão. O objetivo é explorar correlações e identificar padrões que possam auxiliar na compreensão desses dados de forma mais intuitiva e visual.

# Coleta de indicadores
De início precisamos coletar os dados dos indicadores, sendo eles fornecidos pela API do Banco Central do Brasil.

In [ ]:
import pandas as pd
!pip install Python-bcb
from bcb import sgs

def carregar_dados():
    def obter_dado(codigo, nome):
        try:
            df = sgs.get({nome: codigo}, start='2016-01-01')
            df = df.reset_index()
            df.columns = ['Data', nome]
            df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
            return df
        except Exception as e:
            print(f"Erro ao carregar {nome}: {e}")
            return pd.DataFrame(columns=['Data', nome])

    dfSelic = obter_dado(1178, 'Selic')
    dfIpca = obter_dado(433, 'Ipca')
    dfMediaDeflacionada = obter_dado(24381, 'Remuneração Média Deflacionada')
    dfIgpm = obter_dado(189, 'Igpm')
    dfInad = obter_dado(21082, 'Inadimplência')
    dfInadFamilia = obter_dado(29038, 'Inadimplência Familiar')
    dfCredTotal = obter_dado(20631, 'Crédito Total')
    dfDolar = obter_dado(1, 'Dólar')

    # Ajuste Selic para frequência mensal
    if not dfSelic.empty:
        dfSelic = dfSelic.set_index('Data').resample('MS').first().reset_index()

    return dfSelic, dfIpca, dfMediaDeflacionada, dfIgpm, dfInad, dfInadFamilia, dfCredTotal, dfDolar


Neste código criamos uma função na qual devolve um vetor para cada indicador que buscamos. Sendo eles: Selic, Remuneração Média Deflacionada, Igpm, Ipca, Inadimplência, Crédito Total, Dólar.

## Criação dos Gráficos

Criação dos Gráficos
Após a coleta dos dados dos indicadores, começamos a criação dos gráficos para a análise.

In [ ]:
!pip install streamlit
import streamlit as st
import plotly.express as px

class Graficos:
    def exibir_grafico_linha(self, df):
        st.subheader('Gráfico de Linha')
        if df.empty:
            st.error("Não há dados para o gráfico de linha.")
            return
        try:
            fig = px.line(df, x='Data', y=df.columns[1:], title="Evolução dos Indicadores", template='plotly_dark')
            st.plotly_chart(fig, use_container_width=True)
        except Exception as e:
            st.error(f"Erro no gráfico de linha: {e}")

    def exibir_grafico_dispersao(self, df):
        st.subheader('Gráfico de Dispersão')
        if df.empty:
            st.error("Não há dados para o gráfico de dispersão.")
            return
        try:
            col1 = st.selectbox("Indicador X:", df.columns[1:], key="disp1")
            col2 = st.selectbox("Indicador Y:", df.columns[1:], key="disp2")
            fig = px.scatter(df, x=col1, y=col2, title=f"Dispersão: {col1} vs {col2}", template='plotly_dark')
            st.plotly_chart(fig, use_container_width=True)
        except Exception as e:
            st.error(f"Erro no gráfico de dispersão: {e}")

    def exibir_grafico_barras(self, df):
        st.subheader('Gráfico de Barras')
        if df.empty:
            st.error("Não há dados para o gráfico de barras.")
            return
        try:
            col = st.selectbox("Indicador:", df.columns[1:], key="bar")
            fig = px.bar(df, x='Data', y=col, title=f"{col} ao longo do tempo", template='plotly_dark')
            st.plotly_chart(fig, use_container_width=True)
        except Exception as e:
            st.error(f"Erro no gráfico de barras: {e}")

    def exibir_grafico_boxplot(self, df):
        st.subheader('Gráfico de Boxplot')
        if df.empty:
            st.error("Não há dados para o gráfico de boxplot.")
            return
        try:
            col = st.selectbox("Indicador:", df.columns[1:], key="box")
            fig = px.box(df, y=col, title=f"Distribuição de {col}", template='plotly_dark')
            st.plotly_chart(fig, use_container_width=True)
        except Exception as e:
            st.error(f"Erro no gráfico de boxplot: {e}")

    def exibir_matriz_correlacao(self, df):
        st.subheader('Matriz de Correlação de Pearson')
        if df.empty or df.shape[1] <= 2:
            st.info("Selecione pelo menos dois indicadores para gerar a matriz de correlação.")
            return
        try:
            df_corr = df.drop(columns=['Data']).corr(method='pearson')
            fig = px.imshow(
                df_corr,
                text_auto=True,
                color_continuous_scale='RdBu_r',
                title="Correlação de Pearson entre Indicadores",
                aspect='auto',
                template='plotly_dark'
            )
            st.plotly_chart(fig, use_container_width=True)
        except Exception as e:
            st.error(f"Erro ao gerar a matriz de correlação: {e}")


Com esse código criamos os seguites gráficos:

Gráfico de Linha:

*   Mostra a evolução dos indicadores ao longo do tempo, agregando todos os indicadores selecionados para uma visão ampla.


Gráfico de Dispersão:

*   Compara a relação entre dois indicadores, sendo útil para identificar padrões e correlações diretas.


Gráfico de Barras:
* Similar ao gráfico de linha, mas focado em indicadores individuais, permitindo analisar a evolução de um indicador específico.

Gráfico de Boxplot:
* Mostra a dispersão do conjunto de dados, destacando mediana, quartis e possíveis outliers.

Correlação de Pearson:
* Mede a força e a direção da relação linear entre os indicadores, identificando as variáveis mais correlacionadas.

# Gráfico de Regressão

Com o gráfico de regressão linear podemos fazer a predição de dados futuros.

In [5]:
import streamlit as st
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import root_mean_squared_error

class Regressao:
    def exibir_regressao_linear(self, df):
        st.subheader("Regressão Linear Múltipla")

        if df.shape[1] > 2:
            target = st.selectbox("Selecione o indicador a ser previsto (variável dependente):", df.columns[1:], key="reg_target")
            features = st.multiselect("Selecione as variáveis preditoras (independentes):", [col for col in df.columns[1:] if col != target], key="reg_features")

            if features and st.button("Executar Regressão"):
                df_modelo = df.dropna(subset=[target] + features)
                X = df_modelo[features]
                y = df_modelo[target]

                scaler = StandardScaler()
                X_scaled = scaler.fit_transform(X)

                st.markdown("### Verificação de Multicolinearidade (VIF)")
                vif_data = pd.DataFrame()
                vif_data["Variável"] = features
                vif_data["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
                st.dataframe(vif_data)

                st.markdown("### Correlação entre Variáveis Preditivas")
                fig_corr = px.imshow(pd.DataFrame(X, columns=features).corr(), text_auto=True, color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
                st.plotly_chart(fig_corr, use_container_width=True)

                X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
                modelo = LinearRegression()
                modelo.fit(X_train, y_train)
                y_pred = modelo.predict(X_test)

                r2 = r2_score(y_test, y_pred)
                mae = mean_absolute_error(y_test, y_pred)
                rmse = root_mean_squared_error(y_test, y_pred)

                st.markdown(f"### Avaliação do Modelo")
                st.write(f"**R² (coeficiente de determinação):** {r2:.2f}")
                st.write(f"**MAE (erro absoluto médio):** {mae:.2f}")
                st.write(f"**RMSE (raiz do erro quadrático médio):** {rmse:.2f}")

                st.markdown("### Coeficientes do Modelo")
                for var, coef in zip(features, modelo.coef_):
                    st.write(f"- {var}: {coef:.4f}")

                futuro_data = pd.to_datetime(df['Data'].max()) + pd.DateOffset(years=1)
                st.markdown(f"### Previsão para o Ano de {futuro_data.year}")

                features_futuras = X.iloc[-1:].copy()
                features_futuras['Data'] = futuro_data

                X_futuro_scaled = scaler.transform(features_futuras[features])
                y_futuro = modelo.predict(X_futuro_scaled)
                st.write(f"**Previsão do {target} para {futuro_data.year}:** {y_futuro[0]:.2f}")

                fig_pred = px.scatter(
                    x=y_test,
                    y=y_pred,
                    labels={"x": "Valores Reais", "y": "Valores Previsto"},
                    title="Valores Reais vs. Previsto",
                    template="plotly_dark"
                )
                fig_pred.add_shape(
                    type="line",
                    x0=min(y_test),
                    y0=min(y_test),
                    x1=max(y_test),
                    y1=max(y_test),
                    line=dict(color="red", dash="dash")
                )
                st.plotly_chart(fig_pred, use_container_width=True)


Esse código define uma classe chamada Regressao e utiliza a API Sklearn para realizar a regressão linear com machine learning. Além disso é utilizado o  Streamlit para a interface, Pandas para manipulação de dados e Plotly para visualização.

# Apresentação dos gráficos no Dashboard
Agora com os dados coletados e com os gráficos criados, falta montar o Dashboard, na qual vai conter todas essas informações.

In [ ]:
import streamlit as st
import pandas as pd

SELIC = 'Selic'
IPCA = 'IPCA'
REMUNERACAO_DEFLACIONADA = 'Remuneração Média Deflacionada'
IGPM = 'IGPM'
INADIMPLENCIA = 'Inadimplência'
INADIMPLENCIA_FAMILIA = 'Inadimplência Família'
CREDITO_TOTAL = 'Crédito Total'
DOLAR = 'Dólar'

class AppStreamlit:
    def __init__(self):
        self.df_selic, self.df_ipca, self.df_salario, self.df_igpm, self.df_inad, self.df_inad_familia, self.df_cred_total, self.df_dolar = carregar_dados()
        self.graficos = Graficos()
        self.regressao = Regressao()

    def exibir(self):
        st.set_page_config(layout='wide')
        st.title('Painel de Indicadores Econômicos')

        st.sidebar.title("Seleção de Indicadores")
        indicadores = [SELIC, IPCA, REMUNERACAO_DEFLACIONADA, IGPM, INADIMPLENCIA, INADIMPLENCIA_FAMILIA, CREDITO_TOTAL, DOLAR]
        selecionados = st.sidebar.multiselect("Escolha os Indicadores para comparação", indicadores)

        df_base = self.df_selic.copy()
        df_base['Data'] = pd.to_datetime(df_base['Data'], errors='coerce')
        data_minima = df_base['Data'].min().replace(day=1)
        data_maxima = df_base['Data'].max().replace(day=1)

        data_inicial = st.sidebar.date_input("A partir de:", data_minima, min_value=data_minima, max_value=data_maxima)

        if selecionados:
            data_inicial = pd.to_datetime(data_inicial).replace(day=1)
            df = self._preparar_comparacao(selecionados, data_inicial)

            self.graficos.exibir_grafico_linha(df)
            self.graficos.exibir_grafico_dispersao(df)
            self.graficos.exibir_grafico_barras(df)
            self.graficos.exibir_grafico_boxplot(df)
            self.graficos.exibir_matriz_correlacao(df)

            self.regressao.exibir_regressao_linear(df)

    def _preparar_comparacao(self, selecionados, data_inicial):
        df_base = self.df_selic.copy()
        df_base['Data'] = pd.to_datetime(df_base['Data'], errors='coerce')
        df_base = df_base.dropna(subset=['Data']).set_index('Data')
        df_base = df_base[df_base.index >= data_inicial]

        df_comparado = pd.DataFrame(index=df_base.index)

        if SELIC in selecionados:
            df_comparado['Selic'] = df_base['Selic']

        def join_df(indicador, df, coluna):
            if indicador in selecionados and not df.empty:
                temp = df.copy()
                temp['Data'] = pd.to_datetime(temp['Data'], errors='coerce')
                temp.set_index('Data', inplace=True)
                temp = temp[temp.index >= data_inicial]
                return df_comparado.join(temp[[coluna]], how='outer')
            return df_comparado

        df_comparado = join_df(IPCA, self.df_ipca, 'Ipca')
        df_comparado = join_df(REMUNERACAO_DEFLACIONADA, self.df_salario, 'Remuneração Média Deflacionada')
        df_comparado = join_df(IGPM, self.df_igpm, 'Igpm')
        df_comparado = join_df(INADIMPLENCIA, self.df_inad, 'Inadimplência')
        df_comparado = join_df(INADIMPLENCIA_FAMILIA, self.df_inad_familia, 'Inadimplência Familiar')
        df_comparado = join_df(CREDITO_TOTAL, self.df_cred_total, 'Crédito Total')
        df_comparado = join_df(DOLAR, self.df_dolar, 'Dólar')

        df_comparado = df_comparado.dropna(how='all').reset_index()
        return df_comparado

if __name__ == '__main__':
    app = AppStreamlit()
    app.exibir()


Além da apresentação dos gráficos, foi criada uma seleção de indicadores, assim permitindo apenas a visualização dos indicadores da preferência do usuário. Também foi adicionada uma função que filtra os dados a partir do tempo escolhido.

# Páginas Complementares

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

dfs = carregar_dados()

df_merged = dfs[0]
for df in dfs[1:]:
    df_merged = pd.merge(df_merged, df, on="Data", how="inner")

st.title("Resumo e Interpretações de Indicadores Econômicos")

def show_analysis(title, desc, x, y):
    st.markdown(f"### {title}")
    st.markdown(desc)
    fig = px.scatter(df_merged, x=x, y=y, trendline="ols", title=f"{x} vs {y}")
    st.plotly_chart(fig)
    st.markdown("---")
st.markdown("""
## 🧾 Descrição dos Indicadores Econômicos

### 🏦 Selic (Taxa Básica de Juros)
A Selic é a taxa básica de juros da economia brasileira, definida pelo Banco Central. Ela influencia diretamente todas as outras taxas de juros do país, como as cobradas em empréstimos, financiamentos e rendimentos de investimentos. Quando a Selic aumenta, o crédito tende a ficar mais caro, o que reduz o consumo e ajuda a conter a inflação.

### 📈 IPCA (Índice de Preços ao Consumidor Amplo)
É o índice oficial de inflação do Brasil, calculado pelo IBGE. Mede a variação de preços de um conjunto de produtos e serviços consumidos pelas famílias. É usado como referência para metas de inflação do governo.

### 💼 Remuneração Média Deflacionada
Refere-se ao rendimento médio real dos trabalhadores, já descontada a inflação. Um aumento neste indicador significa que o poder de compra da população está aumentando, o que pode impactar o consumo, endividamento e capacidade de pagamento.

### 📊 IGP-M (Índice Geral de Preços – Mercado)
É um índice de inflação calculado pela FGV e amplamente utilizado para reajustes de contratos (como aluguel). Ele reflete a variação de preços em diferentes estágios da economia, como atacado, construção civil e consumidor final.

### ❗ Inadimplência
Mede o percentual de operações de crédito em atraso no sistema financeiro. Altas taxas de inadimplência indicam dificuldades generalizadas da população ou empresas em manter os pagamentos em dia.

### 👨‍👩‍👧‍👦 Inadimplência das Famílias
É um recorte da inadimplência que considera exclusivamente pessoas físicas. Está relacionada com o endividamento familiar, desemprego, inflação e variações na renda.

### 💳 Crédito Total
Representa o total de crédito concedido pelo sistema financeiro, incluindo pessoas físicas e jurídicas. Esse indicador mostra a disponibilidade de capital na economia e o grau de atividade financeira.

### 💵 Dólar
Cotação da moeda americana em relação ao real. O dólar afeta importações, exportações, inflação, investimentos e expectativas do mercado. Seu comportamento está ligado a fatores internos (como política econômica) e externos (como juros dos EUA).

---
""")



# Análises de interesse
show_analysis(
    "📉 Selic vs IGP-M",
    "**Correlação negativa forte (-0.54)**: Alta da Selic tende a conter inflação medida pelo IGP-M.",
    "Selic", "Igpm"
)

show_analysis(
    "💳 Selic vs Inadimplência",
    "**Correlação positiva (0.64)**: Juros maiores tornam o crédito mais caro, aumentando inadimplência.",
    "Selic", "Inadimplência"
)

show_analysis(
    "🏦 Crédito Total vs Inadimplência Familiar",
    "**Correlação fortíssima (0.90)**: Mais crédito disponível aumenta o risco de inadimplência nas famílias.",
    "Crédito Total", "Inadimplência Familiar"
)

show_analysis(
    "💼 Remuneração Média vs Crédito Total",
    "**Correlação moderada (0.29)**: Melhora na renda pode estimular o aumento do crédito.",
    "Remuneração Média Deflacionada", "Crédito Total"
)

show_analysis(
    "💼 Remuneração Média vs Inadimplência Familiar",
    "**Correlação fraca (0.15)**: Pode haver impacto, mas outros fatores como juros e desemprego pesam mais.",
    "Remuneração Média Deflacionada", "Inadimplência Familiar"
)


Essa página tem o propósito de auxiliar o usuário, nela há a descrição de cada Indicador, além disso é apresentada as analises mais comuns e correlacionadas.

In [ ]:
import streamlit as st

st.title("Sobre o Projeto")
st.markdown("""
## Grupo de Análise Econômica



### Integrantes:
- **Beatriz de Souza Santos Rio Branco**
- **Sátiro Gabriel de Souza Santos**
- **Sabrinna Cristina Gomes Vicente**
- **Rodrigo Correa da Gama**
---

Este painel foi desenvolvido como parte de um trabalho acadêmico para explorar indicadores econômicos usando Python e Streamlit.
""")


Página de Créditos, na qual mostra o nome dos criadores do projeto.

# Conclusão
O dashboard da Econovision utiliza de Indicadores Socioeconômicos para criar gráficos que auxiliam na análise de dados, além de ser capaz de fazer previsões futuras a partir de machine learning.

# O dashboard está hospedado em:
#**https://dashboard-bcb.streamlit.app/**